## Goal
Employee turn-over is a very costly problem for companies. The cost of replacing an employee if often larger than 100K USD, taking into account the time spent to interview and ﬁnd a replacement, placement fees, sign-on bonuses and the loss of productivity for several months. 

It is only natural then that data science has started being applied to this area. Understanding why and when employees are most likely to leave can lead to actions to improve employee retention as well as planning new hiring in advance. This application of DS is sometimes called people analytics or people data science (if you see a job title: people data scientist, this is your job). 

In this challenge, you have a data set with info about the employees and have to predict when employees are going to quit by understanding the main drivers of employee churn.


## Challenge  Description
We got employee data from a few companies. We have data about all employees who joined from 2011/01/24 to 2015/12/13. For each employee, we also know if they are still at the company as of 2015/12/13 or they have quit. Beside that, we have general info about the employee, such as avg salary during her tenure, dept, and yrs of experience. 

As said above, the goal is to predict employee retention and understand its main drivers. Speciﬁcally, you should: 
1. Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13. That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have already quit, then company headcount on 2012/03/02 for company 1 would be 1000. You should create a table with 3 columns: day, employee_headcount, company_id. 
2. What are the main factors that drive employee churn? Do they make sense? Explain your ﬁndings. 
3. If you could add to this data set just one variable that could help explain employee churn, what would that be?


In [234]:
import pandas as pd
import numpy as np

In [235]:
data=pd.read_csv("employee_retention_data.csv")

In [236]:
data.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


The meaning of columns:
* employee_id : id of the employee. Unique by employee per company company_id : company id. 
* dept : employee 
* dept seniority : number of yrs of work experience when hired 
* salary: avg yearly salary of the employee during her tenure within the company 
* join_date: when the employee joined the company, it can only be between 2011/01/24 and 2015/12/13 
* quit_date: when the employee left her job (if she is still employed as of 2015/12/13, this ﬁeld is NA)


In [188]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
employee_id    24702 non-null float64
company_id     24702 non-null int64
dept           24702 non-null object
seniority      24702 non-null int64
salary         24702 non-null float64
join_date      24702 non-null object
quit_date      13510 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 1.3+ MB


Convert Pandas Column to DateTime

In [237]:
data['join_date']=pd.to_datetime(data['join_date'])
data['quit_date']=pd.to_datetime(data['quit_date'])

In [191]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
employee_id    24702 non-null float64
company_id     24702 non-null int64
dept           24702 non-null object
seniority      24702 non-null int64
salary         24702 non-null float64
join_date      24702 non-null datetime64[ns]
quit_date      13510 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 1.3+ MB


In [193]:
data.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [194]:
# Check the missing value
data.isnull().sum()

employee_id        0
company_id         0
dept               0
seniority          0
salary             0
join_date          0
quit_date      11192
dtype: int64

## Create a table with 3 columns:
day, employee_headcount, company_id.


In [238]:
data.unique_companies=sorted(data['company_id'].unique())
unique_date = pd.date_range(start='2011-01-24', end='2015-12-13', freq='D')
day = []
company = []
headcount = []
for date in unique_date:
    for idx in unique_companies:
        join=len(data[(data['join_date'] <= date) & (data['company_id'] == idx)])
        quit = len(data[(data['quit_date'] <= date) & (data['company_id'] == idx)])
        day.append(date)
        company.append(company)
        headcount.append(join - quit)
table=pd.DataFrame({'day': day, 'company_id': company, 'employee_headcount': headcount}, 
                     columns=['day', 'company_id', 'employee_headcount'])


C:\Users\blanc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: 

In [219]:
data=data.sort_values('join_date')

In [220]:
data.set_index('join_date', inplace=True)


In [225]:
data

,employee_id,company_id,dept,seniority,salary,quit_date
join_date,,,,,,
2011-01-24,546870.0,8,engineer,14,132000.0,2014-01-31
2011-01-24,901005.0,2,customer_service,21,90000.0,2012-03-23
2011-01-24,375389.0,1,data_science,20,299000.0,2013-03-01
2011-01-24,792687.0,3,engineer,19,283000.0,2012-08-28
2011-01-24,228718.0,2,data_science,10,197000.0,2012-01-18
2011-01-24,281753.0,8,customer_service,5,72000.0,2012-01-06
2011-01-24,527591.0,2,engineer,11,178000.0,2011-12-16
2011-01-24,491696.0,4,data_science,14,127000.0,2014-08-01
2011-01-24,205943.0,8,customer_service,1,26000.0,2012-01-17


In [224]:
data.groupby('company_id','').count()


,employee_id,dept,seniority,salary,quit_date
company_id,,,,,
1,8486,8486,8486,8486,4621
2,4222,4222,4222,4222,2206
3,2749,2749,2749,2749,1531
4,2062,2062,2062,2062,1153
5,1755,1755,1755,1755,983
6,1291,1291,1291,1291,712
7,1224,1224,1224,1224,692
8,1047,1047,1047,1047,579
9,961,961,961,961,529


In [174]:
quit=data.groupby(['quit_date','company_id'])[['company_id']].count()

In [175]:
quit

company_id
quit_date  company_id            
2011-10-13 6                    1
2011-10-14 10                   1
2011-10-21 1                    1
2011-10-28 4                    1
2011-11-11 1                    1
2011-11-22 1                    1
2011-11-25 1                    1
           2                    1
2011-12-02 6                    1
           8                    1
2011-12-09 1                    1
           4                    1
           8                    1
2011-12-13 2                    1
2011-12-16 1                    1
           2                    2
           4                    3
           5                    1
           8                    1
2011-12-23 1                    2
           3                    1
           6                    1
           7                    1
           8                    2
           10                   1
2011-12-30 1                    4
           2                    1
           3                    1
           4                    1
2012-01-03 1                    1
...                           ...
2015-11-25 2                    1
           3                    1
2015-11-27 1                   35
           2                   15
           3                   14
           4                    9
           5                   10
           6                    4
           7                    3
           8                    5
           9                    4
           10                   3
2015-12-01 3                    1
2015-12-03 1                    1
           3                    1
           7                    1
           9                    1
2015-12-04 1                   26
           2                   12
           3                    8
           4                    4
           5                    3
           6                    4
           7                    7
           8                    4
           9                    4
           10                   3
2015-12-08 1                    1
           3                    1
2015-12-09 4                    1

[2724 rows x 1 columns]

In [228]:
join=data.groupby(['join_date','company_id'])[['company_id']]

In [233]:
df = (data.groupby(['company_id'])
                .apply(lambda x: x.set_index('join_date').resample('D').ffill()))

print (df.head())


ValueError: cannot reindex a non-unique index with a method or limit

In [182]:
data.unique_companies=sorted(data['company_id'].unique())
unique_date = pd.date_range(start='2011-01-24', end='2015-12-13', freq='D').to_frame()

In [183]:
pd.merge(unique_date,data)

MergeError: No common columns to perform merge on

In [184]:
unique_date.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_frame'